In [28]:
from scipy import interpolate
from scipy.signal import butter, lfilter, lfilter_zi, find_peaks
import numpy as np 
import pandas as pd 
import glob
from tqdm import tqdm 
import pyarrow.csv

In [37]:
ROOT = "watch_data_test/watch_data_test"
SUBMISSION_SR = 30 

In [52]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

In [53]:
def butter_bandpass_filter_zi(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    zi = lfilter_zi(b, a)
    y,zo = lfilter(b, a, data, zi=zi*data[0])
    return y

In [54]:
def format_peaks_to_intervals(ppg_peaks, ts_start, ts_finish):
    timestamps_interp = np.linspace(ts_start, ts_finish, int((ts_finish - ts_start) * SUBMISSION_SR))
    heartbeat_intervals = np.diff(ppg_peaks, prepend=ts_start)
    func = interpolate.interp1d(ppg_peaks, heartbeat_intervals, "nearest", bounds_error=False, fill_value=0)
    heartbeat_intervals_interp = func(timestamps_interp)
    return heartbeat_intervals_interp

In [55]:
def process_exp(ppg_df, params):
    params['data'] = ppg_df['PPG_signal'].values
    timestamps = ppg_df['Timestamp'].values
    filtered = butter_bandpass_filter_zi(**params)
    ppg_peak_idxs = find_peaks(filtered)[0]
    ppg_peaks = timestamps[ppg_peak_idxs]
    ppg_rr_intervals_interp = format_peaks_to_intervals(ppg_peaks, timestamps[0], timestamps[-1])
    return ppg_rr_intervals_interp

In [56]:
params = {
        'data': None,
        'lowcut': 1.5,
        'highcut': 5,
        'fs': 120,
        'order': 2
}

In [57]:
test_ppgs_pathes = glob.glob(f"{ROOT}/PPG_EXP_*.csv")
test_ppgs_pathes.sort()
all_predictions = []

for path in tqdm(test_ppgs_pathes):
    ppg_df = pyarrow.csv.read_csv(path).to_pandas()
    ppg_rr_intervals_interp = process_exp(ppg_df, params)
    all_predictions.append(ppg_rr_intervals_interp)
all_predictions = np.concatenate(all_predictions)
sample_summission = pd.DataFrame({"PPG_interbeat_interval": all_predictions})
sample_summission['id'] = sample_summission.index
sample_summission.to_csv("submission.csv", index=None)
sample_summission.shape

100%|█████████████████████████████████████████████| 9/9 [00:00<00:00, 44.00it/s]


(1347666, 2)